### 1. 패키지 설치

In [ ]:

%pip install python-docx python-dotenv tiktoken chromadb openai

### 2. Knowledge Base 구성을 위한 데이터 생성
- LangChain의 TextSplitter (RecursiveCharacterTextSplitter등)를 사용할 수 없기 때문에 
- python-docx와 tiktoken을 활용해서 chunk 생성

### 2-1. 문서 가져와서 -> chunk 생성

In [2]:
from docx import Document

document = Document('./tax.docx')

full_text = ''
for index, paragraph in enumerate(document.paragraphs):
    full_text += f'{paragraph.text}\n'

In [3]:
import tiktoken 

def split_text(full_text, chunk_size):
    encoder = tiktoken.encoding_for_model("gpt-4o")
    total_encoding = encoder.encode(full_text)
    total_token_count = len(total_encoding)
    text_list = []
    for i in range(0, total_token_count, chunk_size):
        chunk = total_encoding[i: i+chunk_size]
        decoded = encoder.decode(chunk)
        text_list.append(decoded)
    
    return text_list

In [ ]:
chunk_list = split_text(full_text, 1500)

### 2-2. Chroma에 임베딩 값 저장 

In [5]:
import chromadb

chroma_client = chromadb.Client()

collection_name = 'tax_collection'
tax_collection = chroma_client.create_collection(collection_name)

In [7]:
import os

from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
openai_embedding = OpenAIEmbeddingFunction(api_key=openai_api_key, model_name='text-embedding-3-large')

In [10]:
tax_collection = chroma_client.get_or_create_collection(collection_name, embedding_function=openai_embedding)

ValueError: An embedding function already exists in the collection configuration, and a new one is provided. If this is intentional, please embed documents separately. Embedding function conflict: new: openai vs persisted: default

In [11]:
id_list = []
for index in range(len(chunk_list)):
    id_list.append(f'{index}')

In [12]:

tax_collection.add(documents=chunk_list, ids=id_list)

/Users/lhj/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:31<00:00, 2.64MiB/s]


### 3. 답변 생성을 위한 Retrieval
- Chroma에 저장한 데이터를 유사도 검색(query)를 활용해서 가져옴
- method이름은 query지만 유사도 검색 실행

In [13]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_doc = tax_collection.query(query_texts=query, n_results=3)

In [14]:

retrieved_doc['documents'][0]

['을 말한다)이 속하는 과세기간의 다음 연도 5월 1일부터 5월 31일까지] 대통령령으로 정하는 바에 따라 납세지 관할 세무서장에게 신고하여야 한다. <개정 2017. 12. 19.>\n② 제1항은 해당 과세기간의 과세표준이 없거나 결손금액이 있는 경우에도 적용한다.\n③ 제1항에 따른 양도소득 과세표준의 신고를 확정신고라 한다.\n④ 예정신고를 한 자는 제1항에도 불구하고 해당 소득에 대한 확정신고를 하지 아니할 수 있다. 다만, 해당 과세기간에 누진세율 적용대상 자산에 대한 예정신고를 2회 이상 하는 경우 등으로서 대통령령으로 정하는 경우에는 그러하지 아니하다.\n⑤ 확정신고를 하는 경우 그 신고서에 양도소득금액 계산의 기초가 된 양도가액과 필요경비 계산에 필요한 서류로서 대통령령으로 정하는 것을 납세지 관할 세무서장에게 제출하여야 한다.\n⑥ 납세지 관할 세무서장은 제5항에 따라 제출된 신고서나 그 밖의 서류에 미비한 사항 또는 오류가 있는 경우에는 그 보정을 요구할 수 있다.\n[전문개정 2009. 12. 31.]\n\n제111조(확정신고납부) ① 거주자는 해당 과세기간의 과세표준에 대한 양도소득 산출세액에서 감면세액과 세액공제액을 공제한 금액을 제110조제1항(제118조에 따라 준용되는 제74조제1항부터 제4항까지의 규정을 포함한다)에 따른 확정신고기한까지 대통령령으로 정하는 바에 따라 납세지 관할 세무서, 한국은행 또는 체신관서에 납부하여야 한다.\n② 제1항에 따른 납부를 이 장에서 “확정신고납부”라 한다.<개정 2020. 12. 29.>\n③ 확정신고납부를 하는 경우 제107조에 따른 예정신고 산출세액, 제114조에 따라 결정ㆍ경정한 세액 또는 제82조ㆍ제118조에 따른 수시부과세액이 있을 때에는 이를 공제하여 납부한다.\n[전문개정 2009. 12. 31.]\n\n제112조(양도소득세의 분할납부) 거주자로서 제106조 또는 제111조에 따라 납부할 세액이 각각 1천만원을 초과하는 자는 대통령령으로 정하는 바에 따라 그 납부할 세액의 일부를 납


### 4. Augmentation을 위한 Prompt 활용 및 답변 생성
- Retrieval된 데이터는 system prompt에 추가해서 LLM의 배경지식으로 활용
- Large Language Model은 사용자 질문에 대해 system prompt에 있는 배경지식을 활용하여 답변 생성

In [15]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": f"당신은 한국의 소득세 전문가 입니다. 아래 내용을 참고해서 사용자의 질문에 답변해주세요 {retrieved_doc['documents'][0]}"},
    {"role": "user", "content": query}
  ]
)

In [16]:
response.choices[0].message.content

'연봉 5천만 원인 직장인의 소득세를 계산하려면, 한국의 종합소득세율을 적용해야 합니다. 2023년 기준 소득세율은 다음과 같습니다:\n\n- 1,200만 원 이하: 6%\n- 1,200만 원 초과 ~ 4,600만 원 이하: 15%\n- 4,600만 원 초과 ~ 8,800만 원 이하: 24%\n- 8,800만 원 초과 ~ 1억 5천만 원 이하: 35%\n- 1억 5천만 원 초과 ~ 3억 원 이하: 38%\n- 3억 원 초과 ~ 5억 원 이하: 40%\n- 5억 원 초과: 42%\n\n연봉 5천만 원에 대해서는 다음과 같이 계산됩니다.\n\n1. 1,200만 원까지: 1,200만 원 × 6% = 72만 원\n2. 1,200만 원 초과 ~ 4,600만 원: (4,600만 원 - 1,200만 원) × 15% = 510만 원\n3. 4,600만 원 초과 ~ 5,000만 원: (5,000만 원 - 4,600만 원) × 24% = 96만 원\n\n각 구간의 세액을 합산하여 총 소득세를 계산하면 다음과 같습니다:\n\n72만 원 + 510만 원 + 96만 원 = 678만 원\n\n따라서, 연봉 5천만 원인 직장인의 소득세는 약 678만 원입니다.\n\n여기에 기본공제 및 기타 소득공제, 세액공제가 적용되면 실제 납부할 세액은 달라지므로, 자세한 내용은 국세청 연말정산 간소화 서비스나 세무사와 상담하여 구체적인 공제를 고려한 세액을 알아보시기 바랍니다.'